In [1]:
import numpy as np
from pathlib import Path
import cv2
import open3d as o3d

In [2]:
inverted_matrix_path = "PATH TO INVERTED MATRIX" # "./data/output/02801_mk29/matrix_xyz_02801_mk29.npy"
depth_map_img_path = "PATH TO DEPTH MAP CHARACTER" # "./data/output/02801_mk29/character_xyz/depth/0.png"

In [3]:
def read_files(inverted_matrix_path, depth_map_img_path, d_size=None):
    depth_img = cv2.imread(depth_map_img_path, 0)
    if d_size is None:
        d_size = 512
    resize_depth_img = cv2.resize(depth_img, (d_size, d_size))
    inverted_matrix = np.load(inverted_matrix_path)
    return resize_depth_img, inverted_matrix

In [4]:
def convert_depth_to_pc(depth_img, inverted_matrix):
    width, height = depth_img.shape
    x_range = np.arange(width)
    y_range = np.arange(height)
    x_mesh, y_mesh = np.meshgrid(x_range, y_range)
    x_mesh_flat = x_mesh.reshape((-1,))
    y_mesh_flat = y_mesh.reshape((-1,))
    z_mesh = depth_img[y_mesh_flat, x_mesh_flat]
    
    x_mesh_flat = x_mesh_flat.reshape(-1, 1)
    y_mesh_flat = y_mesh_flat.reshape(-1, 1)
    z_mesh = z_mesh.reshape(-1, 1)
    np_ones = np.ones((x_mesh_flat.shape[0], 1))
    
    homo_points = np.hstack([x_mesh_flat, y_mesh_flat, z_mesh, np_ones])
    pc_points = (inverted_matrix@homo_points.T).T
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc_points[:, :3])
    # o3d.io.write_point_cloud("sync.ply", pcd)
    return pcd

In [5]:
resize_depth_img, inverted_matrix = read_files(inverted_matrix_path, depth_map_img_path)
pcd = convert_depth_to_pc(resize_depth_img, inverted_matrix)

In [ ]:
o3d.visualization.draw_geometries([pcd])